In [1]:
from torchvision import datasets
from torchvision.transforms import ToTensor

In [2]:
train_data = datasets.MNIST(
    root="data",
    train=True,
    transform=ToTensor(),
    download=True
)

test_data = datasets.MNIST(
    root="data",
    train=False,
    transform=ToTensor(),
    download=True
)

In [3]:
train_data

Dataset MNIST
    Number of datapoints: 60000
    Root location: data
    Split: Train
    StandardTransform
Transform: ToTensor()

In [4]:
test_data

Dataset MNIST
    Number of datapoints: 10000
    Root location: data
    Split: Test
    StandardTransform
Transform: ToTensor()

In [5]:
train_data.data.shape

torch.Size([60000, 28, 28])

In [6]:
train_data.targets

tensor([5, 0, 4,  ..., 5, 6, 8])

In [7]:
from torch.utils.data import DataLoader

loaders = {
    'train': DataLoader(train_data,
                        batch_size=64,
                        shuffle=True,
                        num_workers=1),
    'test': DataLoader(train_data,
                        batch_size=64,
                        shuffle=True,
                        num_workers=1),                    
}

In [8]:
loaders

{'train': <torch.utils.data.dataloader.DataLoader at 0x7f75e546cd90>,
 'test': <torch.utils.data.dataloader.DataLoader at 0x7f75e546d000>}

In [9]:
from model_architecture import MyCNN
import torch.nn as nn
import torch.optim as optim

In [10]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = MyCNN().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
loss_function = nn.CrossEntropyLoss()

def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(loaders['train']):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = loss_function(output, target)
        loss.backward()
        optimizer.step()

        if batch_idx % 20 == 0:
            print(f"Training epoch: {epoch} [{batch_idx * len(data)}/{len(loaders['train'].dataset)} ({100.0 * batch_idx / len(loaders['train']):.0f}%)]\t{loss.item():.6f}")

In [11]:
def test():
    model.eval()

    test_loss = 0
    correct = 0

    with torch.no_grad():
        for data, target in loaders['test']:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += loss_function(output, target).item()

            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
    
    test_loss /= len(loaders['test'].dataset)
    print(f"\nTest set: Average loss: {test_loss: .6f}, Accuracy: {correct}/{len(loaders['test'].dataset)} ({100.0 * correct / len(loaders['test'].dataset):.0f}%)\n")

In [12]:
for epoch in range(1, 11):
    train(epoch)
    test()

/d/WSL/handwritten_digit_recognition/model_architecture.py:22: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.softmax(x)


Training epoch: 1 [0/60000 (0%)]	2.302941
Training epoch: 1 [1280/60000 (2%)]	2.297723


Training epoch: 1 [2560/60000 (4%)]	2.213227
Training epoch: 1 [3840/60000 (6%)]	2.014501
Training epoch: 1 [5120/60000 (9%)]	1.758603
Training epoch: 1 [6400/60000 (11%)]	1.854520
Training epoch: 1 [7680/60000 (13%)]	1.851071
Training epoch: 1 [8960/60000 (15%)]	1.722142
Training epoch: 1 [10240/60000 (17%)]	1.750216
Training epoch: 1 [11520/60000 (19%)]	1.666988
Training epoch: 1 [12800/60000 (21%)]	1.769713
Training epoch: 1 [14080/60000 (23%)]	1.793087
Training epoch: 1 [15360/60000 (26%)]	1.691888
Training epoch: 1 [16640/60000 (28%)]	1.731554
Training epoch: 1 [17920/60000 (30%)]	1.714721
Training epoch: 1 [19200/60000 (32%)]	1.670842
Training epoch: 1 [20480/60000 (34%)]	1.690089
Training epoch: 1 [21760/60000 (36%)]	1.725609
Training epoch: 1 [23040/60000 (38%)]	1.693843
Training epoch: 1 [24320/60000 (41%)]	1.638272
Training epoch: 1 [25600/60000 (43%)]	1.626231
Training epoch: 1 [26880/60000 (45%)]	1.687175
Training epoch: 1 [28160/60000 (47%)]	1.632195
Training epoch: 1 [294

In [14]:
torch.save(model, "model.pt")

In [15]:
model.parameters

<bound method Module.parameters of MyCNN(
  (conv1): Conv2d(1, 10, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(10, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2_dropout): Dropout2d(p=0.5, inplace=False)
  (fc1): Linear(in_features=320, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=10, bias=True)
)>